In [1]:
### Loading Credentials from local file; this cell is meant to be deleted before publishing
import yaml

with open("../creds.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

uri = cfg["sonar_creds"]["uri"]
user = cfg["sonar_creds"]["user"]
password = cfg["sonar_creds"]["pass"]

# Project Summary 


## Citing SoNAR (IDH)

# Data Description


The SoNAR (IDH) database consists of nodes and edges. Each of the nodes and edges have additional attributes that provide rich meta information. 

This data description section provides details about the data sources and overall characteristics of the data. The section is based on the state of the SoNAR (IDH) database during February 2021. 

More in depth details about the data, the database and the data preparation will get published soon. 

## Summary Stats

The SoNAR (IDH) database has the following aggregated characteristics:

**Nodes Summary**

* 9 categories of Nodes
* 34.511.952 Nodes

|Node Type | Node Count |
|---------------|------------|
| CorpName      | 1.487.711  |
| GeoName       | 308.197    |
| MeetName      | 814.044    |
| PerName       | 5.087.660  |
| TopicTerm     | 212.135    |
| UniTitle      | 385.300    |
| ChronTerm     | 537.054    |
| IsilTerm      | 611        |
| Resource      | 25.679.240 |

**Edges Summary**

* 98.530.160 Edges
* 9 categories of Edges


| Edge Type       | Edge Count     |
|---------------------|------------|
| RelationToPerName   | 14.630.465 |
| RelationToCorpName  | 5.099.190  |
| RelationToMeetName  | 263.180    |
| RelationToUniTitle  | 53.998     |
| RelationToTopicTerm | 4.951.617  |
| RelationToGeoName   | 5.140.556  |
| RelationToChronTerm | 5.446.841  |
| RelationToIsil      | 55.556.913 |
| RelationToResource  | 7.387.400  |
| socialRelation      | 40.301.595 |

## Data Sources 

SoNAR (IDH) combines data from four different data sources. The table below provides a compact overview:


|Data Source |  Number of Nodes | Number of Edges <br>(incl. *RelationToIsilTerm*) |
|--------|----------| -------- |
|[GND (*Integrated Authority File*)](https://www.dnb.de/EN/Professionell/Standardisierung/GND/gnd_node.html)| 8.295.047 | 32.776.628 |
|[DNB (*German National Library*)](https://www.dnb.de/EN/Home/home_node.html)|19.384.733| 5.655.859 |
|[ZDB (*Zeitschriftendatenbank*)](https://www.zeitschriftendatenbank.de/startseite/)|1.908.334| 43.419.339 |
|KPE|4.386.173| 16.678.334 |

# Data Access

We will need some specific libraries to work with the SoNAR (IDH) database. Let's start with installing the `neo4j` library.

When you are using the curriculum on binder or locally or by running it as a docker container locally, the pacckage is already installed. When you want to interact with the SoNAR (IDH) database independently install the package with the following code line in a new notebook cell:

```python
!pip install neo4j
```

In [17]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri, auth=(user, password))

With the call above we create a [Neo4j driver object](https://neo4j.com/docs/api/python-driver/current/api.html#driver). This driver now stores the connection details for the database. We can use this driver now to send requests to the database for example to request data.

# Data Exploration

Data exploration is usually the very first thing to do when working with new data. So let's start diving into the SoNAR (IDH) database by exploring it. 

Whenever we want to retrieve data from the Neo4j database of SoNAR (IDH) we can use a query language called "Cypher Query Language". Cypher provides a rather easy to comprehend syntax for requesting data from the database.

Throughout this curriculum we will use this Cypher Query Language whenever we directly retrieve data from SoNAR (IDH).

## Nodes

### Node Labels

Let's start off with a simple query. Let's request the database to return all [node labels](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-labels). Node labels are basically categories the nodes can belong to. You can think of them as entity groups. The SoNAR (IDH) database distinguishes between Persons, Corporations and more. Let's ask the database it self to return all the labels available. 

In [5]:
with driver.session() as session:
    result = session.run("CALL db.labels()").data()
    
result

[{'label': 'CorpName'},
 {'label': 'GeoName'},
 {'label': 'MeetName'},
 {'label': 'PerName'},
 {'label': 'TopicTerm'},
 {'label': 'UniTitle'},
 {'label': 'ChronTerm'},
 {'label': 'IsilTerm'},
 {'label': 'Resource'}]

**Code Breakdown:**

>The `with` statement is basically used to make the database call as resource effective and concise as possible. There are more advantages of the `with` call but their explanation would exceed the goal of this curriculum. However, an in-depth explanation of the `with` statement can be found [here](https://www.python.org/dev/peps/pep-0343/).
>
>When we request data from the database we need to establish a connection (`session`). The `driver` object we created earlier stores the connection details. When we use the method `driver.session()` we establish a new connection. This connection is assigned to the object `session` object for the `while` statement.
>
>The most relevant part of the code for retrieving the data is `"CALL db.labels()"`. This part is the actual Cypher query. The `CALL` clause is used to call the `db.labels()` procedure. More details about Neo4j procedures can be found below.
>
>The result of this code chunk is a list that contains a key-value pair (`dictionary`) per label in the database.  

Some useful built-in procedures for exploring and describing the database are listed in the table below. You can get a full list of built-in procedures by using the following query: `CALL dbms.procedures()`


|Procedure | Description |
|---------|----------|
|`db.labels()`| List all labels in the database.|
|`db.propertyKeys()`|List all property keys in the database.|
|`db.relationshipTypes`|List all relationship types in the database. |
|`db.schema`| Show the schema of the data. |
|`db.stats.retrieve`|Retrieve statistical data about the current database. <br>Valid sections are 'GRAPH COUNTS', 'TOKENS', 'QUERIES', 'META'|

### Selecting Nodes

You can select nodes by using the `MATCH` statement. Cypher uses `ASCII-art` syntax to define nodes, relationships and the direction of relationships in queries. 

Nodes are referred to by using parentheses `()`. Inside the parentheses you can define a Node variable. This variable can be used to refer to a specific set of nodes throughout the rest of the query.

The example below matches any kind of Node and assigns the variable name n `(n)`. We use the `LIMIT` statement to tell the database we only want to have the first 10 results.

In [6]:
query = """
MATCH (n)
RETURN n
LIMIT 5
"""

In [7]:
import pandas as pd

with driver.session() as session:
    result = session.run(query).data()
    
result

[{'n': {'GenType': 'b',
   'DateOriginal': '1951-',
   'SpecType': 'kiz',
   'VariantName': 'CNS;;;CNS',
   'DateApproxBegin': '1951',
   'Id': '(DE-588)110-7',
   'id': 'Aut110_7',
   'Uri': 'http://d-nb.info/gnd/110-7',
   'Name': 'Congress of Neurological Surgeons'}},
 {'n': {'GenType': 'b',
   'Id': '(DE-588)191-0',
   'id': 'Aut191_0',
   'Uri': 'http://d-nb.info/gnd/191-0',
   'SubUnit': 'División de Ciencias Matemáticas, Médicas y de la Naturaleza',
   'Name': 'Consejo Superior de Investigaciones Científicas'}},
 {'n': {'GenType': 'b',
   'Id': '(DE-588)257-4',
   'id': 'Aut257_4',
   'SpecType': 'kiz',
   'Uri': 'http://d-nb.info/gnd/257-4',
   'Name': 'Fondazione Antonio Baselli'}},
 {'n': {'GenType': 'b',
   'Id': '(DE-588)273-2',
   'id': 'Aut273_2',
   'SpecType': 'wit',
   'Uri': 'http://d-nb.info/gnd/273-2',
   'Name': 'Copyright Law Symposium'}},
 {'n': {'GenType': 'b',
   'DateOriginal': '1865-',
   'VariantName': "Kornel'skii Universitet;;;Cornell Univ.;;;Universit

This output looks good, but let's populate a data frame with this output, so it's easier to work with later on. 

The code cell below populates a pandas data frame with a loop - this is not a recommended way to solve this data transformation since it is very slow. Later on in this curriculum we will use more efficient ways to get this job done.

In [8]:
df = pd.DataFrame()

for i in range(len(result)):
    temp_df = pd.DataFrame.from_dict(result[i], orient = "index")
    df = pd.concat([df, temp_df], axis = 0, ignore_index = True) 
df

,GenType,DateOriginal,SpecType,VariantName,DateApproxBegin,Id,id,Uri,Name,SubUnit,OldId
0,b,1951-,kiz,CNS;;;CNS,1951,(DE-588)110-7,Aut110_7,http://d-nb.info/gnd/110-7,Congress of Neurological Surgeons,NaN,NaN
1,b,NaN,NaN,NaN,NaN,(DE-588)191-0,Aut191_0,http://d-nb.info/gnd/191-0,Consejo Superior de Investigaciones Científicas,"División de Ciencias Matemáticas, Médicas y...",NaN
2,b,NaN,kiz,NaN,NaN,(DE-588)257-4,Aut257_4,http://d-nb.info/gnd/257-4,Fondazione Antonio Baselli,NaN,NaN
3,b,NaN,wit,NaN,NaN,(DE-588)273-2,Aut273_2,http://d-nb.info/gnd/273-2,Copyright Law Symposium,NaN,NaN
4,b,1865-,kiz,Kornel'skii Universitet;;;Cornell Univ.;;;Univ...,1865,(DE-588)275-6,Aut275_6,http://d-nb.info/gnd/275-6,Cornell University,NaN,(DE-588)1088112064;;;(DE-588)1088112544;;;(DE-...


### Filtering Nodes

In [9]:
query = """
MATCH (n:PerName)
RETURN n
LIMIT 5"""

In [10]:
import pandas as pd

with driver.session() as session:
    result = session.run(query).data()
    
df = pd.DataFrame()

for i in range(len(result)):
    temp_df = pd.DataFrame.from_dict(result[i], orient = "index")
    df = pd.concat([df, temp_df], axis = 0, ignore_index = True) 
df

,GenType,VariantName,SpecType,Id,id,Uri,Name,DateApproxEnd,DateStrictOriginal,DateStrictBegin,DateStrictEnd,DateApproxOriginal,Gender,DateApproxBegin,OldId
0,p,"Lombez, Ambrosius de;;;La Peirie, Ambroise;;;L...",piz,(DE-588)100000096,Aut100000096,http://d-nb.info/gnd/100000096,Ambrosius,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p,"Bauer, Johanes Gottfried;;;Bauerus, Johannes G...",piz,(DE-588)100000193,Aut100000193,http://d-nb.info/gnd/100000193,"Bauer, Johann Gottfried",1763,20.02.1695-02.03.1763,20.02.1695,02.03.1763,1695-1763,1,1695,NaN
2,p,"Baur, W.;;;Baur, Johannes Guilielmus;;;Baur, W...",piz,(DE-588)100000231,Aut100000231,http://d-nb.info/gnd/100000231,"Baur, Johann Wilhelm",1642,31.05.1600-XX.01.1642,31.05.1600,XX.01.1642,1607-1642,1,1607,(DE-588)17418154X;;;(DE-588)172001064
3,p,"Amman, Reinerius",piz,(DE-588)100000355,Aut100000355,http://d-nb.info/gnd/100000355,"Amman, Reiner",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,p,"Mosca, Gioseppe",piz,(DE-588)100000541,Aut100000541,http://d-nb.info/gnd/100000541,"Mosca, Giuseppe",1839,NaN,NaN,NaN,1772-1839,NaN,1772,NaN


**Filtering Nodes by Properties**

In [11]:
query = """
MATCH (n {Name: 'Weber, Max'})
RETURN n
LIMIT 10"""

In [105]:
with driver.session() as session:
    print(session.run(query).data())

[{'n': {'GenType': 'p', 'SpecType': 'piz', 'Id': '(DE-588)117167665', 'id': 'Aut117167665', 'Gender': '1', 'Uri': 'http://d-nb.info/gnd/117167665', 'Name': 'Weber, Max'}}, {'n': {'GenType': 'p', 'DateApproxEnd': '1937', 'VariantName': 'Weber, Max W.;;;Weber, Max Carl Wilhelm;;;Weber, Max C. W.;;;Weber, Max Wilhelm Carl;;;Weber, M.', 'SpecType': 'piz', 'DateApproxBegin': '1852', 'DateApproxOriginal': '1852-1937', 'Gender': '1', 'Id': '(DE-588)117167681', 'id': 'Aut117167681', 'Uri': 'http://d-nb.info/gnd/117167681', 'Name': 'Weber, Max'}}, {'n': {'GenType': 'p', 'DateApproxEnd': '1897', 'SpecType': 'piz', 'DateApproxBegin': '1836', 'Id': '(DE-588)117167738', 'DateApproxOriginal': '1836-1897', 'Gender': '1', 'id': 'Aut117167738', 'Uri': 'http://d-nb.info/gnd/117167738', 'Name': 'Weber, Max'}}, {'n': {'GenType': 'p', 'DateApproxEnd': '1948', 'DateStrictOriginal': '20.09.1874-30.09.1948', 'SpecType': 'piz', 'DateStrictBegin': '20.09.1874', 'DateStrictEnd': '30.09.1948', 'OldId': '(DE-588)1

More details on properties can be found in the [properties section](#Properties).

## Relationship Types

Similar to node labels we can retrieve the categories of the relations inside the database. Every relation must have exactly one relationship type. This type defines the kind or category the relation belongs to. 

In [18]:
with driver.session() as session:
    result = session.run("CALL db.relationshipTypes()").data()

    
pd.DataFrame(result)

,relationshipType
0,RelationToIsilTerm
1,RelationToChronTerm
2,RelationToCorpName
3,RelationToTopicTerm
4,RelationToGeoName
5,SocialRelation
6,RelationToMeetName
7,RelationToPerName
8,RelationToUniTitle
9,RelationToResource


## Properties

Properties are additional information that might be assigned to nodes or relationships. Properties can provide meta information e.g. about geographic locations, names, gender and pretty much anything else that might be relevant as an information for nodes or relationships. 

Also, properties can be used to identify a specific subset of nodes or relationships. 

In [14]:
#query = """
#CALL db.propertyKeys() YIELD propertyKey AS prop
#MATCH (n)
#WHERE n[prop] IS NOT NULL
#RETURN prop, count(n) AS numNodes
#"""


#with driver.session() as session:
#    result = session.run(query).data()
    
#result

## Cypher Query Language

## General Database Summary

In [20]:
query = """
match (n) return 'Number of Nodes: ' + count(n) as output UNION
match ()-[]->() return 'Number of Relationships: ' + count(*) as output UNION
CALL db.labels() YIELD label RETURN 'Number of Labels: ' + count(*) AS output UNION
CALL db.relationshipTypes() YIELD relationshipType  RETURN 'Number of Relationship Types: ' + count(*) AS output
"""

with driver.session() as session:
    result = session.run(query).data()
    
pd.DataFrame(result)


,output
0,Number of Nodes: 46831805
1,Number of Relationships: 191363660
2,Number of Labels: 9
3,Number of Relationship Types: 10


# Descriptive Analyses

## Summarise Node Labels

In [21]:
result = {"label": [], "count": []}

with driver.session() as session:
    labels = [row["label"] for row in session.run("CALL db.labels()")]
    for label in labels:
        query = f"MATCH (:{label}) RETURN count(*) as count"
        count = session.run(query).single()["count"]
        result["label"].append(label)
        result["count"].append(count)
        
node_labels_df = pd.DataFrame(result)
node_labels_df.sort_values(by = "count")

,label,count
7,IsilTerm,611
4,TopicTerm,212135
1,GeoName,308197
5,UniTitle,385300
6,ChronTerm,537054
2,MeetName,814044
0,CorpName,1487711
3,PerName,5087660
8,Resource,37999093


## Summarise Relationship Types


In [22]:
result = {"relType": [], "count": []}

with driver.session() as session:
    rel_types = [row["relationshipType"] for row in session.run("CALL db.relationshipTypes()")]
    for rel_type in rel_types:
        query = f"MATCH ()-[:{rel_type}]->() RETURN count(*) as count"
        count = session.run(query).single()["count"]
        result["relType"].append(rel_type)
        result["count"].append(count)
        
rel_type_df = pd.DataFrame(result)
rel_type_df.sort_values(by = "count")

,relType,count
8,RelationToUniTitle,128256
6,RelationToMeetName,422333
1,RelationToChronTerm,5446841
2,RelationToCorpName,6728127
4,RelationToGeoName,6861379
9,RelationToResource,7389423
7,RelationToPerName,20857782
3,RelationToTopicTerm,24068056
5,SocialRelation,40301595
0,RelationToIsilTerm,79159868


# Complex Queries & Data Preparation